## Helper Utils

In [14]:
import string
import itertools
import numpy as np 
def generate_strings(length):
    chars = ["A","C","G","T"]
    return ("".join(item) for item in itertools.product(chars, repeat=length))

def getKmers(s, k):
    for i in range(1 + len(s) - k):
        yield s[i:i+k]

def HammingDistance(p, q):
    count = 0
    for (i,j) in zip(p,q):
        if i != j:
            count += 1
    return count

def NeighborhoodofString(pattern, d):
    if d == 0:
        return [pattern]
    if len(pattern) == 1:
        return ['A', 'C', 'G', 'T']
    neighborhood = []
    sufneigh =NeighborhoodofString(pattern[1:],d)
    for x in sufneigh:
        if HammingDistance(pattern[1:],x) < d:
            neighborhood.extend(['A'+x, 'C'+x, 'G'+x, 'T'+x])
        else:
            neighborhood.append(pattern[0] + x)
    return neighborhood

### (2A) Implement MOTIFENUMERATION

In [4]:
def motifEnumeration(k, d, DNA):
    pattern = set()
    for combo in generate_strings(k): #gets combinations
        if all(any(HammingDistance(combo, pat) <= d for pat in getKmers(string, k)) for string in DNA):
            pattern.add(combo)
    return pattern 

motifEnumeration(3, 1,
["ATTTGGC",
"TGCCTTA",
"CGGTATC",
"GAAAATT"])

{'ATA', 'ATT', 'GTT', 'TTT'}

### (2B) Find a Median String

In [7]:
def DiffBetweenPatternandStrings(pattern, dna): # minimizes the sum of hamming distances to the given dnas with a pattern
    k =  len(pattern)
    diff = 0;
    for d in dna:
        ham_distance = 2 ** 31
        for i in range (len(d) - k + 1):
            pattern_ = d[i : i+k]
            if (HammingDistance(pattern,pattern_) < ham_distance):
                ham_distance = HammingDistance(pattern,pattern_)
        diff = diff + ham_distance

    return diff

def MedianString(k, dna) : # A string that minimizes the sum of distances to the strings of a given set 
    diff = 2 ** 31 
    kmers = generate_strings(k)
    median = ""
    for kmer in kmers:
        temp_diff =  DiffBetweenPatternandStrings(kmer, dna)
        if (temp_diff < diff) :
            diff =  temp_diff
            median = kmer

    return median

MedianString(3,
["AAATTGACGCAT",
"GACGACCACGTT",
"CGTCAGCGCCTG",
"GCTGAGCACCGG",
"AGTACGGGACAG"])

'ACG'

### (2C) Find a Profile-most Probable k-mer in a String

In [8]:
def calculateProbability(pattern, profile_mat): # given a profile, calculates the probability of a string occuring 
    temp_pr = 1.0
    for j in range(len(pattern)):
        if(pattern[j] == "A"):
            temp_pr  = temp_pr * profile_mat ["A"][j]
        elif(pattern[j] == "C"):
            temp_pr  = temp_pr * profile_mat ["C"][j]
        elif(pattern[j] == "G"):
            temp_pr  = temp_pr * profile_mat ["G"][j]
        elif(pattern[j] == "T"):
            temp_pr  = temp_pr * profile_mat ["T"][j]
    return temp_pr


def ProfileMostProbableKmer(text, k, profile):
    max_p = -1
    most_probable_kmer = ''
    for i in range(len(text)-k+1):
        pattern = text[i: i+k]
        p = calculateProbability(pattern, profile)
        if p > max_p:
            max_p = p
            most_probable_kmer = pattern           
    return most_probable_kmer

### (2D) Implement GREEDYMOTIFSEARCH

In [10]:
def Count(Motifs):
    count = {} # initializing the 4xk count dictionary
    k = len(Motifs[0])
    count = {'A':[0]*k,'C':[0]*k,'G':[0]*k,'T':[0]*k}
    
    t = len(Motifs)
    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            count[symbol][j] += 1
    return count

def Consensus(Motifs): # get the consensus string for given motifs by picking the most frequent symbol in each column
    k = len(Motifs[0])
    count = Count(Motifs)
    consensus = ""
    for j in range(k):
        m = 0
        frequentSymbol = ""
        for symbol in "ACGT":
            if count[symbol][j] > m:
                m = count[symbol][j]
                frequentSymbol = symbol
        consensus += frequentSymbol
    return consensus

def Profile(Motifs): ## generate profile matrix from motifs
    t = len(Motifs)
    k = len(Motifs[0])
    profile = Count(Motifs)
    print("Count: \n", profile)

    for i in profile:  
        for j in range(k):
           profile[i][j] = float(profile[i][j])/float(t)   
    return profile

def Score(Motifs):
    consensus = Consensus(Motifs) #consensus string
    counts = Count(Motifs) # count matrix
    score = len(Motifs)*len(consensus) 
    i = 0
    for symbol in consensus:  
        score -= counts[symbol][i] 
        i += 1
    return score 

def GreedyMotifSearch(Dna, k, t):
    BestMotifs = []
    for i in range(0, t):
        BestMotifs.append(Dna[i][0:k]) # first k-mers from each of the dna strings
    
    n = len(Dna[0]) 
    for i in range(n-k+1):
        Motifs = []
        Motifs.append(Dna[0][i:i+k]) # kmer from 1st string
        for j in range(1, t): #for rest of the strings
            P = Profile(Motifs[0:j]) # successively calculates profile matrix by adding one string per iteration
            print("intmed Profile for ", Motifs[0:j])
            print(P)
            Motifs.append(ProfileMostProbableKmer(Dna[j], k, P))
            print("PMP kmer for ",Dna[j]," is: ",ProfileMostProbableKmer(Dna[j], k, P))

        print("current motifs: ", Motifs)
        if Score(Motifs) < Score(BestMotifs):
            BestMotifs = Motifs
        print("..........................................................................................")
    return BestMotifs

GreedyMotifSearch(["ACCTAT", "AGTTGG", "CCGTTC", "ACGTAA"], 4, 4)

Count: 
 {'A': [1, 0, 0, 0], 'C': [0, 1, 1, 0], 'G': [0, 0, 0, 0], 'T': [0, 0, 0, 1]}
intmed Profile for  ['ACCT']
{'A': [1.0, 0.0, 0.0, 0.0], 'C': [0.0, 1.0, 1.0, 0.0], 'G': [0.0, 0.0, 0.0, 0.0], 'T': [0.0, 0.0, 0.0, 1.0]}
PMP kmer for  AGTTGG  is:  AGTT
Count: 
 {'A': [2, 0, 0, 0], 'C': [0, 1, 1, 0], 'G': [0, 1, 0, 0], 'T': [0, 0, 1, 2]}
intmed Profile for  ['ACCT', 'AGTT']
{'A': [1.0, 0.0, 0.0, 0.0], 'C': [0.0, 0.5, 0.5, 0.0], 'G': [0.0, 0.5, 0.0, 0.0], 'T': [0.0, 0.0, 0.5, 1.0]}
PMP kmer for  CCGTTC  is:  CCGT
Count: 
 {'A': [2, 0, 0, 0], 'C': [1, 2, 1, 0], 'G': [0, 1, 1, 0], 'T': [0, 0, 1, 3]}
intmed Profile for  ['ACCT', 'AGTT', 'CCGT']
{'A': [0.6666666666666666, 0.0, 0.0, 0.0], 'C': [0.3333333333333333, 0.6666666666666666, 0.3333333333333333, 0.0], 'G': [0.0, 0.3333333333333333, 0.3333333333333333, 0.0], 'T': [0.0, 0.0, 0.3333333333333333, 1.0]}
PMP kmer for  ACGTAA  is:  ACGT
current motifs:  ['ACCT', 'AGTT', 'CCGT', 'ACGT']
.....................................................

['ACCT', 'AGTT', 'CCGT', 'ACGT']

### (2E) Implement GREEDYMOTIFSEARCH with Pseudocounts

In [21]:
def CountWithPseudo(Motifs):
    count = {}
    k = len(Motifs[0])
    for symbol in "ACGT":
        count[symbol] = []
        for j in range(k):
            count[symbol].append(1)
    t = len(Motifs)
    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            count[symbol][j] += 1 
    return count

def ProfileWithPseudoCount(Motifs,pseudocount):
    t = len(Motifs)
    k = len(Motifs[0])
    count = CountWithPseudo(Motifs)
    profile = {}
    for letter in count.keys():
        profile[letter] = [float(x)/ (t+ (pseudocount*4)) for x in count[letter]]
    return profile  

def GreedyMotifSearchWithPseudocounts( k, t, Dna,):
    BestMotifs = []
    pseudocount = 1
    for i in range(0, t):
        BestMotifs.append(Dna[i][0:k]) 
    
    n = len(Dna[0]) 
    for i in range(n-k+1):
        Motifs = []
        Motifs.append(Dna[0][i:i+k])
        for j in range(1, t):
            P = ProfileWithPseudoCount(Motifs[0:j],pseudocount) 
            Motifs.append(ProfileMostProbableKmer(Dna[j], k, P))
    
        if Score(Motifs) < Score(BestMotifs):
            BestMotifs = Motifs
    return BestMotifs

GreedyMotifSearchWithPseudocounts(3, 5,
["GGCGTTCAGGCA",
"AAGAATCAGTCA",
"CAAGGAGTTCGC",
"CACGTCAATCAC",
"CAATAATATTCG"])


['TTC', 'ATC', 'TTC', 'ATC', 'TTC']

### (2F) Implement RANDOMIZEDMOTIFSEARCH

In [16]:
def MotifFromProfile(Dna, k, profile):
    motifs=[]
    for i in range(len(Dna)):
        motifs.append(ProfileMostProbableKmer(Dna[i],k,profile))
    return motifs

def RandomMotifs(Dna, k):
    result = []
    for string in Dna:
        index = np.random.randint(0, len(string) - k)
        result.append(string[index:index + k])
    return result

def RandomizedMotifSearch(Dna,k, t):
    motifs = RandomMotifs(Dna,k)
    bestMotifs = motifs
    while True:
        profile_mat = Profile(motifs)
        motifs = MotifFromProfile(Dna,k,profile_mat)
        if Score(motifs) < Score(bestMotifs):
            bestMotifs = motifs
        else:
            return bestMotifs

RandomizedMotifSearch(
["CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA",
"GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG",
"TAGTACCGAGACCGAAAGAAGTATACAGGCGT",
"TAGATCAAGTTTCAGGTGCACGTCGGTGAACC",
"AATCCACCAGCTCCACGTGCAATGTTGGCCTA"],
8, 5)

Count: 
 {'A': [2, 0, 1, 2, 2, 3, 3, 1], 'C': [2, 3, 1, 0, 1, 0, 1, 0], 'G': [0, 2, 1, 3, 1, 2, 0, 3], 'T': [1, 0, 2, 0, 1, 0, 1, 1]}
Count: 
 {'A': [2, 0, 0, 1, 2, 4, 4, 1], 'C': [1, 2, 1, 0, 1, 0, 1, 0], 'G': [0, 3, 1, 4, 0, 1, 0, 3], 'T': [2, 0, 3, 0, 2, 0, 0, 1]}


['AGTAAACG', 'TGTGTAAG', 'ACCGAAAG', 'TCGGTGAA', 'CGTGCAAT']

### (2G) Implement GIBBSSAMPLER

In [19]:
def Profile_randomly_generated_kmer(Text,k,Profile):
    
    for i in range(len(Text) - k + 1):
        return kmer

def GibbsSampler(Dna,k,t,N):
    motifs = RandomMotifs(Dna,k)
    bestMotifs = motifs
    for run in range(N):
        i = np.random.randint(t)
        motifs_except_ith = np.delete(motifs,i)
        profile_mat = Profile(motifs_except_ith)
        motifs[i] = ProfileMostProbableKmer(Dna[i],k,profile_mat)
        if Score(motifs) < Score(bestMotifs):
            bestMotifs = motifs
    return bestMotifs

GibbsSampler(
["CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA",
"GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG",
"TAGTACCGAGACCGAAAGAAGTATACAGGCGT",
"TAGATCAAGTTTCAGGTGCACGTCGGTGAACC",
"AATCCACCAGCTCCACGTGCAATGTTGGCCTA"],
8, 5, 100)

Count: 
 {'A': [3, 1, 1, 0, 0, 0, 2, 2], 'C': [0, 0, 0, 1, 1, 2, 2, 1], 'G': [0, 3, 1, 0, 2, 2, 0, 1], 'T': [1, 0, 2, 3, 1, 0, 0, 0]}
Count: 
 {'A': [3, 2, 2, 0, 0, 0, 1, 2], 'C': [0, 0, 0, 2, 1, 1, 3, 2], 'G': [0, 2, 1, 0, 3, 3, 0, 0], 'T': [1, 0, 1, 2, 0, 0, 0, 0]}
Count: 
 {'A': [3, 1, 2, 0, 0, 0, 2, 1], 'C': [0, 0, 0, 2, 1, 1, 2, 3], 'G': [0, 3, 1, 0, 3, 3, 0, 0], 'T': [1, 0, 1, 2, 0, 0, 0, 0]}
Count: 
 {'A': [3, 1, 2, 0, 0, 0, 1, 2], 'C': [0, 0, 0, 2, 1, 1, 3, 2], 'G': [0, 3, 1, 0, 3, 3, 0, 0], 'T': [1, 0, 1, 2, 0, 0, 0, 0]}
Count: 
 {'A': [4, 1, 2, 0, 0, 0, 2, 2], 'C': [0, 0, 0, 2, 1, 2, 2, 2], 'G': [0, 3, 2, 0, 3, 2, 0, 0], 'T': [0, 0, 0, 2, 0, 0, 0, 0]}
Count: 
 {'A': [4, 2, 2, 0, 0, 1, 2, 1], 'C': [0, 0, 0, 3, 2, 1, 2, 3], 'G': [0, 2, 1, 0, 2, 2, 0, 0], 'T': [0, 0, 1, 1, 0, 0, 0, 0]}
Count: 
 {'A': [4, 2, 2, 0, 0, 0, 2, 2], 'C': [0, 0, 0, 2, 1, 2, 2, 2], 'G': [0, 2, 2, 0, 3, 2, 0, 0], 'T': [0, 0, 0, 2, 0, 0, 0, 0]}
Count: 
 {'A': [4, 2, 2, 0, 0, 1, 2, 1], 'C': [0, 0, 0, 3, 2, 

['AAACGGCC', 'AAGTGCCA', 'AGACCGAA', 'AGGTGCAC', 'AATCCACC']

### (2H) Implement DISTANCEBETWEENPATTERNANDSTRINGS

In [20]:
def DiffBetweenPatternandStrings(pattern, dna): # minimizes the sum of hamming distances to the given dnas with a pattern
    k =  len(pattern)
    diff = 0;
    for d in dna:
        ham_distance = 2 ** 31
        for i in range (len(d) - k + 1):
            pattern_ = d[i : i+k]
            if (HammingDistance(pattern,pattern_) < ham_distance):
                ham_distance = HammingDistance(pattern,pattern_)
        diff = diff + ham_distance

    return diff